In [1]:
from groq import Groq

class Chat:
    def __init__(self,
                api_key_path = "./api_key",
                model="llama3-8b-8192", 
                temperature=0,
                max_tokens=2048,
                top_p=1,
                stop=None,
                stream=False,
                ):
        # groq parameters
        self.api_key_path = api_key_path
        self.model = model

        # model parameters
        self.__temperature = temperature
        self.__max_tokens = max_tokens
        self.__top_p = top_p
        self.__stop = stop
        self.__stream = stream

        # read api key
        self.read_api_key()

    def read_api_key(self):
        with open(self.api_key_path, "r") as f:
            api_key = f.readline()
        self.client = Groq(api_key=api_key)

    def __construct_messages__(self, user_msgs, system_msg=None, assistant_msgs=None):
        messages = []

        if system_msg:
            msg = {
                "role" : "system",
                "content" : system_msg
            }
            messages.append(msg)

        while user_msgs:
            cur_user_msg = user_msgs.pop(0)
            user_msg = {
                "role" : "user",
                "content" : cur_user_msg
            }
            messages.append(user_msg)

            if assistant_msgs:
                cur_assisant_msg = assistant_msgs.pop(0)
                assistant_msg = {
                    "role" : "assistant",
                    "content" : cur_assisant_msg
                }
                messages.append(assistant_msg)

        return messages
            

    def chat(self, 
            user_msgs:list[str], 
            system_msg:str = None, 
            assistant_msgs:list[str] = None,
            _temperature:float=None,
            _max_tokens:int=None,
            _top_p:float=None,
            _stop:str=None,
            _stream:bool=None):
        
        # set defaults
        _temperatue = _temperature if _temperature is not None else self.__temperature
        _max_tokens = _max_tokens if _max_tokens is not None else self.__max_tokens
        _top_p = _top_p if _top_p is not None else self.__top_p
        _stop = _stop if _stop is not None else self.__stop
        _stream = _stream if _stream is not None else self.__stream

        # construct the messages
        messages = self.__construct_messages__(user_msgs=user_msgs, system_msg=system_msg, assistant_msgs=assistant_msgs)
        
        # get the chat completion
        chat_completion = self.client.chat.completions.create(
            messages=messages,
            model=self.model,
            temperature=_temperatue,
            max_tokens=_max_tokens,
            top_p=_top_p,
            stop=_stop,
            stream=_stream
        )

        # retrive the outputs
        __reason_for_stop__ = chat_completion.choices[0].finish_reason
        output = chat_completion.choices[0].message.content

        # handle exceptions
        if __reason_for_stop__ != "stop":
            print("[COLCA-WARNING] The chat is stopped because of token limit exceeded : ", __reason_for_stop__)

        return output

c = Chat()
c.chat(["hi, how are you?"])


"Hi! I'm just an AI, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or tasks you may have! How about you, how's your day going?"

In [22]:
from functools import reduce

def wrap(string):
        # string = reduce(lambda x, y : x+y, string)
        string = string.replace("  ", "")
        string = string.replace("\t\n", "\t")
        string = string.replace("\t", "")
        return string

class Prompts:
    class Summarize:
        SYSTEM_MESSAGE = wrap("""You are an AI summarizer, your job is to summarize the piece of text based on the given title \
                        and give a short hand form of them with the important keywords in it. Make sure to format the summarized text \
                        in such a way that an AI model can understand and can use it as a prompt. Do Not include any words other the summarized \
                        text. Do not include any unnecessary lines in the response""")
        
        GUARD_RAILS = wrap("""Summarize the above titles and their content. Make sure to include only the summarized text and nothing else
                                    1. Extract only core information directly relevant to the title or topic.
                                    2. Prioritize main points, essential keywords, and relevant details.
                                    3. Avoid additional explanations, background context, or filler words.
                                    4. Include only high-value, specific keywords (e.g., names, locations, numbers, technical terms).
                                    5. Exclude redundant or less relevant terms that do not contribute to the prompt's focus.
                                    6. Use bullet points or brief statements when possible.
                                    7. Maintain concise, structured phrasing, avoiding full sentences if not necessary.
                                    8. Avoid adding extra formatting or punctuation unless it enhances clarity for an AI model.
                                    9. Ensure that the final output can be understood by an AI model as a standalone prompt.
                                    10. Avoid words like "summary," "in summary," or other introductory or concluding phrases.
                                    11. Aim for brevity; convey information in the shortest, most direct form possible.
                                    12. Use abbreviations and shorthand when appropriate and easily understandable by an AI model.
                                    13. Ensure that the summarized text accurately reflects the original text's intent and meaning.
                                    14. Avoid altering or misinterpreting any critical information or nuances.
                                    15. No extra instructions, explanations, or framing text around the response.
                                    16. Avoid system-generated phrases or additional prompts beyond the summarization requirements.
                                   """)
        
        QA_SYSTEM_MSG = wrap("""You are an AI summarizer, your job is to summarize the lead information along with the questions they asked and the answer generated \
                             by an AI system. Make sure to format the summarized text in such a way that it easy for a sales manager to understand the history of conversation \
                             between the AI system and the lead who asked the actual answers. Do not include any words other the summarized text. Do not include any \
                             unnecessary lines in the response. Your summary should definitely include the questions asked and the response generated, you can list them out.""")
        
    class QA:
        QUESTION_JUDGE = wrap("""You are an AI assistant, your job is to validate a question asked by a user based on the provided context. \
                                      If the question is not relevant to the context at hand, respond "invalid", if its relevant then respond "valid". \
                                      Do not incldue any other words except "valid" or "invalid" in your response. The question will be provided by the user \
                                      so act as a judge in this scenario \n\nContext: {context} \n\nQuestion: {question} \n\nResponse: """)
        
        DEFAULT_RESPONSE = "Sorry, I cannot help you with that"

        QUESTION_REPLY = wrap("""You are an AI salesperson with specialized knowledge limited to the provided context. \
                                    Your responses should strictly adhere to the information given and refrain from any outside knowledge, assumptions, or invented details. \
                                    When responding to a potential lead: 
                                    Answer questions concisely, only drawing from the provided context. 
                                    Avoid any phrases or context that extend beyond the exact information provided. 
                                    If a question requires knowledge outside of the given facts, respond with: '"""+ DEFAULT_RESPONSE + """'. \
                                    Avoid adding any additional wording or elaboration.
                                    \n\nContext: {context}\n\nQuestion: {question}\n\nReply: """)

P = Prompts 
print(P.QA.QUESTION_REPLY.format(context="this is the context", question="this is the question"))

You are an AI salesperson with specialized knowledge limited to the provided context. Your responses should strictly adhere to the information given and refrain from any outside knowledge, assumptions, or invented details. When responding to a potential lead: 
Answer questions concisely, only drawing from the provided context. 
Avoid any phrases or context that extend beyond the exact information provided. 
If a question requires knowledge outside of the given facts, respond with: 'Sorry, I cannot help you with that'. Avoid adding any additional wording or elaboration.


Context: this is the context

Question: this is the question

Reply: 


In [23]:
class Summarizer:
    def __init__(self,
                chat:Chat,
                ):
        self.chat = chat
    
    def summarize_email_product(self, documents:dict,):
        # retrive the necessary prompts
        sys_msg = Prompts.Summarize.SYSTEM_MESSAGE
        guard_rails = Prompts.Summarize.GUARD_RAILS

        # create the message content
        message = ""
        for title, content in documents.items():
            cur_msg = f"title : {title}\n\n"
            cur_msg += f"content : {content}\n\n"
            message += cur_msg

        message += guard_rails

        # pass it through the chat
        output = self.chat.chat(system_msg=sys_msg,
                                user_msgs=[message])
        
        return output
    
    def summarize_qa_lead(self, qa:tuple[str, str], lead_info: str):
        # retrive the necessary prompts
        sys_msg = Prompts.Summarize.QA_SYSTEM_MSG
        guard_rails = Prompts.Summarize.GUARD_RAILS

        # create the message content
        message = "Lead Info : " + lead_info + "\n\n"
        
        for question, response in qa:
            if response == Prompts.QA.DEFAULT_RESPONSE:
                continue
            message += "Question Asked by Lead : " + question + "\nAI Generated Response : " + response + "\n\n"

        message += guard_rails

        # pass it through the chat
        output = self.chat.chat(system_msg=sys_msg,
                                user_msgs=[message])
        
        return output



In [24]:
class Sample:
    def __init__(self):
        with open("./sample/sample_company_information.txt", "r") as f:
            self.product_info = f.read()

        with open("./sample/sample_email_content.txt", "r") as f:
            self.email_content = f.read()

        with open("./sample/sample_lead_info.txt", "r") as f:
            self.lead_info = f.read()


class ReplyMachine:
    def __init__(self, 
                product_information: str,
                email_content: str,
                chat: Chat,
                lead_info: str,
                ):      
        self.chat = chat
        self.product_information = product_information
        self.email_content = email_content
        self.lead_info = lead_info
        self.text_summarized_email_product = None
        self.questions_asked = []
        self.qa_threads = []

    def summarize_email_and_product(self):
        # initialize a summarizer
        summarizer = Summarizer(self.chat)

        # create document dict
        documents = {}
        documents["Product Information"] = self.product_information
        documents["Email Content"] = self.email_content

        # summarize
        summarized_text = summarizer.summarize_email_product(documents=documents)
        self.text_summarized_email_product = summarized_text
        print("[INFO] Summarized Email Content and Product Information")

    def __summarize_qa_and_lead_info(self):
        # initialize the summarizer
        summarizer = Summarizer(self.chat)

        # summarize
        summarized_text = summarizer.summarize_qa_lead(qa = self.qa_threads, lead_info=self.lead_info)
        return summarized_text

        

    def __validate_question(self, question):
        # get the prompt
        message = Prompts.QA.QUESTION_JUDGE.format(question=question, context=self.product_information)

        # generate a valid/invalid for the question
        output = self.chat.chat(user_msgs=[message])

        if "valid" in output.split(" "):
            return True
        return False


    def reply(self, question):
        
        # generate product information and email summary if not generated already
        if self.text_summarized_email_product is None:
            self.summarize_email_and_product()

        # log the questions
        self.questions_asked.append(question)

        # validate the question
        validation = self.__validate_question(question)
        
        if not validation:
            print("[COLCA-ERROR] The asked question is not valid : ", question)
            self.qa_threads.append((question,Prompts.QA.DEFAULT_RESPONSE))
            return Prompts.QA.DEFAULT_RESPONSE

        # get the prompt
        message = Prompts.QA.QUESTION_REPLY.format(question=question, context=self.text_summarized_email_product)

        # generate the output from the model
        output = self.chat.chat(user_msgs=[message])

        # add it to the thread
        self.qa_threads.append((question, output))
       
        return output
    
    def quick_bytes(self):
        summaried_qa_lead = self.__summarize_qa_and_lead_info()
        return summaried_qa_lead

        



In [25]:
sample = Sample()
chat = Chat()

R = ReplyMachine(
    product_information=sample.product_info,
    email_content=sample.email_content,
    chat=chat,
    lead_info=sample.lead_info
)


In [29]:
question = "Hi, How are you?"
print("-"*20, " Question ", "-"*20)
print(question)
print("ans : ")
print(R.reply(question))

question = "How many cameras does iphone 15 pro has?"
print("-"*20, " Question ", "-"*20)
print(question)
print("ans : ")
print(R.reply(question))

question = "How many usb ports does the iphone 15 pro has?"
print("-"*20, " Question ", "-"*20)
print(question)
print("ans : ")
print(R.reply(question))



--------------------  Question  --------------------
Hi, How are you?
ans : 
[COLCA-ERROR] The asked question is not valid :  Hi, How are you?
Sorry, I cannot help you with that
--------------------  Question  --------------------
How many cameras does iphone 15 pro has?
ans : 
The iPhone 15 Pro has 3 cameras: 48MP wide sensor, 12MP telephoto lens, and 12MP ultra-wide lens.
--------------------  Question  --------------------
How many usb ports does the iphone 15 pro has?
ans : 
The iPhone 15 Pro has 1 USB-C port.


In [30]:
print(R.qa_threads)

print(R.quick_bytes())

[('Hi, How are you?', 'Sorry, I cannot help you with that'), ('How many cameras does iphone 15 pro has?', 'The iPhone 15 Pro has 3 cameras: 48MP wide sensor, 12MP telephoto lens, and 12MP ultra-wide lens.'), ('How many usb ports does the iphone 15 pro has?', 'The iPhone 15 Pro has 1 USB-C port.'), ('Hi, How are you?', 'Sorry, I cannot help you with that'), ('How many cameras does iphone 15 pro has?', 'The iPhone 15 Pro has 3 cameras: a 48MP wide sensor, a 12MP telephoto lens, and a 12MP ultra-wide lens.'), ('How many usb ports does the iphone 15 pro has?', 'The iPhone 15 Pro has 1 USB-C port.'), ('Hi, How are you?', 'Sorry, I cannot help you with that'), ('How many cameras does iphone 15 pro has?', 'The iPhone 15 Pro has 3 cameras: 48MP wide sensor, 12MP telephoto lens, and 12MP ultra-wide lens.'), ('How many usb ports does the iphone 15 pro has?', 'The iPhone 15 Pro has 1 USB-C port.')]
Lead Information:
• Name: John Doe
• Profession: Tech Enthusiast / IT Professional
• Location: Indi

In [1]:
import streamlit as st

# Title of the app
st.title('My First Streamlit App')

# Display a simple text
st.write('Hello, welcome to my first Streamlit app!')

# Input widget
name = st.text_input('Enter your name')

# Button widget
if st.button('Say Hello'):
    st.write(f'Hello, {name}!')
    
# Add a slider
age = st.slider('Select your age', 0, 100, 25)

# Display the age
st.write(f'Your age is: {age}')

2024-11-10 04:01:50.621 
  command:

    streamlit run /Users/aditya/Library/Python/3.10/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]
